# 오픈 데이터 추가

- KorQuAD 1.0의 전체 데이터는 1,560 개의 Wikipedia article에 대해 10,645 건의 문단과 66,181 개의 질의응답 쌍으로, Training set 60,407 개, Dev set 5,774 개의 질의응답쌍으로 구분하였습니다.

- KorQuAD 1.0의 데이터셋은 CC BY-ND 2.0 KR 라이센스를 따릅니다.

- 《 》answers에 추가

- 고대그리스어, 상형 문자 제거

- train Dataset만 사용, context 1개에 여러 question이 있는 데이터인데, context마다 random question 1개만 추가

In [1]:
from datasets import load_dataset
from tqdm import tqdm
import numpy as np

/data/ephemeral/home/mrc-nlp-08/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
dataset = load_dataset("KorQuAD/squad_kor_v1")

In [41]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [4]:
#train_dataset = dataset['train']

In [42]:
from datasets import concatenate_datasets, DatasetDict
dataset = concatenate_datasets([dataset['train'], dataset['validation']])
train_dataset = DatasetDict({'train': dataset})['train']
train_dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 66181
})

In [43]:
df_train = train_dataset.to_pandas()

In [44]:
import re
import unicodedata
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

def preprocess_text(text):
    # 유니코드 BOM 제거
    text = text.lstrip('\ufeff')
    
    # 하이퍼링크 제거 (http, https, www 형식의 링크 제거)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)
    
    # 여러 개의 공백을 하나의 공백으로 변환
    text = re.sub(r'\s+', ' ', text)
    
    # 유니코드 문자가 정상적으로 인코딩되었는지 확인 후 정규화
    text = unicodedata.normalize('NFC', text)
    
    # 양쪽 끝 공백 제거
    text = text.strip()

    # words = text.split()
    # filtered_words = []

    # # 한글,영어 외 문자 제거
    # for word in words:
    #     if re.fullmatch(r'[가-힣0-9]+', word):
    #         filtered_words.append(word)
    #         continue

    #     if re.fullmatch(r'[a-zA-Z0-9]+', word):
    #         try:
    #             lang = detect(word)
    #             if lang == 'en':
    #                 filtered_words.append(word)
    #         except LangDetectException:
    #             continue
    #         continue

    #     cleaned_word = re.sub(r'[^\w\s\(\)\《\》]', '', word)
    #     if re.fullmatch(r'[가-힣a-zA-Z0-9\(\)\《\》]+', cleaned_word):
    #         filtered_words.append(cleaned_word)

    # text = " ".join(filtered_words)
    return text

# 예시 텍스트
# text = '''\ufeff안녕하세요! <a href="https://example.com">여기를 클릭하세요</a> www.example.com에서 더 많은 정보를 확인하세요.'''
# clean_text = preprocess_text(text)

# print(clean_text)

In [45]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gogamza/kobart-base-v2")  # BertTokenizer

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [46]:
print('before preprocess',len(train_dataset))

before preprocess 66181


# Train dataset filtering

In [47]:
long_tokenized_pass=[]
answer_result_pass=[]
not_in_pre_context_pass=[]
html_answer_pass=[]

In [48]:
train_filter_lst = []
for _, row in tqdm(df_train.iterrows(),total=df_train.shape[0]):
    context = row['context']
    preprocess_context = preprocess_text(context)

    # 1500보다 긴 token 수 예외
    if len(tokenizer.encode(preprocess_context)) > 2000:
        long_tokenized_pass.append(1)
        continue
        
    answer = row['answers']
    text = answer['text'][0]
    # answer가 다 것 " . 우 문 으로 끝나면 예외
    if text[-1] in ['다', '것', '"', '.', '우', '문']:
        answer_result_pass.append(1)
        continue
    
    answer_start_fix = preprocess_context.find(text)
    find_text = preprocess_context[answer_start_fix:answer_start_fix+len(text)]

            
    # find_text : preprocess_context 에서 [start index ~ start index+len(text)] 까지의 값 
    # (preproces과정이 답에 영향을 미치는 경우)
    # answer랑 다르면 예외
    if text != find_text:
        not_in_pre_context_pass.append(1)
        continue

    # # answer에 《 》추가 및 answer_start_fix 1 감소
    # if answer_start_fix!=0 and answer_start_fix+len(text)<len(preprocess_context):
    #     if preprocess_context[answer_start_fix-1]=="《" and preprocess_context[answer_start_fix+len(text)]=="》":
    #         text ="《"+find_text+"》"
    #         answer_start_fix-=1

    
    # answer에 html_answer_start 있으면 예외
    try:
        del answer['html_answer_start']
        html_answer_pass.append(1)
    except:
        pass
    
    answer['text'] = [text]
    answer['answer_start'] = np.array([answer_start_fix])
    row['context'] = preprocess_context
    row['answers'] = answer
    row['html'] = None
    row['id'] = 'KorQuAD 1.0_' + str(row['id'])
    train_filter_lst.append(row)

  0%|          | 0/66181 [00:00<?, ?it/s]

100%|██████████| 66181/66181 [01:24<00:00, 782.11it/s]


In [49]:
print(len(long_tokenized_pass))
print(len(answer_result_pass))
print(len(not_in_pre_context_pass))
print(len(html_answer_pass))

print('total',len(long_tokenized_pass)+len(answer_result_pass)+len(not_in_pre_context_pass)+len(html_answer_pass))

8
686
160
0
total 854


In [50]:
print('filtered_result : ',len(train_filter_lst))

filtered_result :  65327


In [51]:
import pandas as pd
df_train_filter = pd.DataFrame(train_filter_lst)

In [52]:
df_train_filter

,id,title,context,question,answers,html
0,KorQuAD 1.0_6566495-0-0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,"{'text': ['교향곡'], 'answer_start': [54]}",None
1,KorQuAD 1.0_6566495-0-1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,"{'text': ['1악장'], 'answer_start': [421]}",None
2,KorQuAD 1.0_6566495-0-2,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,"{'text': ['베토벤의 교향곡 9번'], 'answer_start': [194]}",None
3,KorQuAD 1.0_6566518-0-0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,"{'text': ['파우스트'], 'answer_start': [15]}",None
4,KorQuAD 1.0_6566518-0-1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,"{'text': ['합창교향곡'], 'answer_start': [354]}",None
...,...,...,...,...,...,...
66176,KorQuAD 1.0_6511152-4-0,삼성_갤럭시_S7_엣지,2016년 9월 4일 중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생했다. ...,중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생한 년도는?,"{'text': ['2016년'], 'answer_start': [0]}",None
66177,KorQuAD 1.0_6511152-4-1,삼성_갤럭시_S7_엣지,2016년 9월 4일 중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생했다. ...,갤럭시 S7엣지를 만든 회사는?,"{'text': ['삼성전자'], 'answer_start': [83]}",None
66178,KorQuAD 1.0_6511152-4-2,삼성_갤럭시_S7_엣지,2016년 9월 4일 중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생했다. ...,2016년 9월 4일 갤럭시 S7엣지가 폭발한 사건이 발생한 지역은?,"{'text': ['중동 지역'], 'answer_start': [12]}",None
66179,KorQuAD 1.0_6535617-4-0,삼성_갤럭시_S7_엣지,2016년 9월 4일 중동 지역에서 갤럭시 S7 엣지가 폭발하는 사건이 발생했다. ...,갤럭시 노트 7은 출시 며칠만에 기기 결함으로 터지기 시작하였나?,"{'text': ['18일'], 'answer_start': [286]}",None


In [53]:
def sampling_func(data):
    np.random.seed(104)
    N = len(data)
    sample_n = 1 # integer
    sample = data.take(np.random.permutation(N)[:sample_n])
    return sample

In [54]:
df_train_filter = df_train_filter.groupby('context').apply(sampling_func)
df_train_filter = df_train_filter.reset_index(drop=True)

/tmp/ipykernel_1286336/2650914660.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_train_filter = df_train_filter.groupby('context').apply(sampling_func)


In [16]:
# df_train_filter.to_csv('koquadv1_train.csv', encoding='utf-8-sig')

In [55]:
dataset.save_to_disk('./resources/data/only_kosquadv1_train_dataset')

Saving the dataset (1/1 shards): 100%|██████████| 66181/66181 [00:00<00:00, 523519.70 examples/s]


# 기존 Train + KorQuAD Train

In [16]:
from datasets import load_from_disk
datasets = load_from_disk("./resources/data/train_dataset") # 기존 train dataset

In [17]:
datasets

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})

In [18]:
raw_train_df = datasets['train'].to_pandas()

In [19]:
df_train_filter = df_train_filter.drop(['html'], axis=1)
raw_train_df_augmentation = pd.concat([raw_train_df, df_train_filter])
raw_train_df_augmentation = raw_train_df_augmentation.reset_index(drop=True)

In [20]:
raw_train_df_augmentation

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293.0,42.0
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638.0,2873.0
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028.0,230.0
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146.0,992.0
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334.0,548.0
...,...,...,...,...,...,...,...
13551,에리히_폰_만슈타인,히틀러가 자신에게 다른 보직을 맡기지 않을 것임이 명백해지자 만슈타인은 1944년 ...,1944년 만슈타인은 어느 곳에 부동산을 샀는가?,KorQuAD 1.0_6586570-31-0,"{'text': ['포메라니아'], 'answer_start': [50]}",NaN,NaN
13552,에리히_폰_만슈타인,히틀러는 하리코프 탈환의 정치적 중요성을 더하기 위해 3월 10일 몸소 전선을 방문...,만슈타인은 언제 하리코프를 재탈환 했는가?,KorQuAD 1.0_6586570-24-0,"{'text': ['3월 14일'], 'answer_start': [50]}",NaN,NaN
13553,헬,힐다 엘리스 데이비드슨은 1948년에 보존된 문헌들에 나타나는 “여신으로서의” 헬이...,헬은 무엇의 여신인가?,KorQuAD 1.0_6571447-4-1,"{'text': ['죽음의 여신'], 'answer_start': [215]}",NaN,NaN
13554,가리온2,힙합 가수로서 살아가는 이야기를 강렬한 비트와 감각 있는 가사로 풀어내어 평단의 극...,한국대중음악상에서 최다 후보인 6개 부문에 이름을 올린 그룹의 이름은 무엇인가?,KorQuAD 1.0_6471732-5-1,"{'text': ['브로콜리 너마저'], 'answer_start': [112]}",NaN,NaN


In [21]:
from datasets import Dataset

In [22]:
train_dataset = Dataset.from_pandas(raw_train_df_augmentation, preserve_index=False)

# Valid는 증강안함

In [23]:
validation_dataset=datasets['validation']

# Final output

In [24]:
from datasets import DatasetDict

In [25]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset
})

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 13556
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})


In [32]:
dataset_dict.save_to_disk('./resources/data/data_kosquadv1_train_dataset')

Saving the dataset (1/1 shards): 100%|██████████| 240/240 [00:00<00:00, 45806.01 examples/s]


In [26]:
temp = load_from_disk('./resources/data/data_kosquadv1_train_dataset')
temp

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 13556
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})